In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.FilterSamples import FilterSamples

import numpy as np
import polars as pl
import datetime
import random
import matplotlib.pyplot as plt

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
file_handler = logging.FileHandler(f"notebook-optuna-{formatted_date}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-07-12 23:23:25,377 - This will print to the notebook's output cell


In [2]:
params = {
    "idxAfterPrediction": 5,
    'timesteps': 60,
    'target_option': 'last',
    
    "TreeTime_lstm_units": 64,
    "TreeTime_lstm_num_layers": 3,
    "TreeTime_lstm_dropout": 0.00001,
    "TreeTime_lstm_recurrent_dropout": 0.00001,
    "TreeTime_lstm_learning_rate": 0.001,
    "TreeTime_lstm_optimizer": "adam",
    "TreeTime_lstm_bidirectional": True,
    "TreeTime_lstm_batch_size": 2**14,
    "TreeTime_lstm_epochs": 4,
    "TreeTime_lstm_l1": 0.00001,
    "TreeTime_lstm_l2": 0.00001,
    "TreeTime_inter_dropout": 0.00001,
    "TreeTime_input_gaussian_noise": 0.00001,
    "TreeTime_lstm_conv1d": True,
    "TreeTime_lstm_conv1d_kernel_size": 3,
    "TreeTime_lstm_loss": "mse",
}

In [3]:
stock_group = "group_snp500_finanTo2011"

eval_date = datetime.date(2025,  2,  1)

start_Date = datetime.date(year=2014, month=1, day=1)

ls = LoadupSamples(
    train_start_date=start_Date,
    test_dates=[eval_date],
    group=stock_group,
    params=params,
)

In [4]:
ls.load_samples(main_path="../src/featureAlchemy/bin/")

2025-07-12 23:24:49,590 - Test date 2025-02-01 not found in the database. Resetting to last trading day.


In [5]:
dates = ls.meta_pl_train.select(pl.col("date")).to_series()
Xtree_all = ls.train_Xtree
ytree_all = ls.train_ytree
        
def est_samples(X_all, y_all, dates: pl.Series, end_train_date, train_days, test_days):
    train_start = end_train_date - datetime.timedelta(days=train_days)
    train_end   = end_train_date

    test_start  = end_train_date + datetime.timedelta(days=1)
    test_end    = end_train_date + datetime.timedelta(days=test_days)

    # slice train
    train_mask = (
        (dates >= train_start) &
        (dates <= train_end)
    ).fill_null(False).to_numpy()

    # slice test
    test_mask = (
        (dates >= test_start) &
        (dates <= test_end)
    ).fill_null(False).to_numpy()

    return (
        X_all[train_mask],
        X_all[test_mask],
        y_all[train_mask],
        y_all[test_mask],
        dates.filter(train_mask),
        dates.filter(test_mask)
    )

In [6]:
import optuna
logger.setLevel(logging.INFO)
def objective(trial: optuna.Trial) -> float:
    # Suggest hyperparameters
    opt_params = {
        "FilterSamples_lincomb_q_up": 0.95,
        "FilterSamples_lincomb_lr": trial.suggest_float("lincomb_lr", 5e-5, 1e-2, log=True),
        "FilterSamples_lincomb_epochs": trial.suggest_int("lincomb_epochs", 300, 1000, step = 100),
        "FilterSamples_lincomb_probs_noise_std": trial.suggest_float("lincomb_noise_std", 0.005, 0.2, log=True),
        "FilterSamples_lincomb_subsample_ratio": trial.suggest_float("lincomb_subsample_ratio", 0.05, 0.5),
        "FilterSamples_lincomb_sharpness": trial.suggest_float("lincomb_sharpness", 0.2, 2.5, log=True),
        "FilterSamples_lincomb_featureratio": trial.suggest_float("lincomb_featureratio", 0.05, 0.99, log=True),
        "FilterSamples_lincomb_itermax": 1,
        "FilterSamples_lincomb_show_progress": False,
    }
    
    end_train_dates = sorted([
        datetime.date(2024,  7,  1) - datetime.timedelta(days=i*120 + random.randint(-10,10)) 
        for i in range(15)
    ])
    
    train_days = trial.suggest_int("num_train_days", 180, 600, step=30)  # 6 months to 2 years of training data
    test_days = 30

    # Build and evaluate FilterSamples for each instance
    scores_train = []
    scores_test = []
    for i, end_date in enumerate(end_train_dates):
        try:
            (
                Xtree_train,
                Xtree_test,
                ytree_train,
                ytree_test,
                samples_dates_train,
                samples_dates_test
            ) = est_samples(Xtree_all, ytree_all, dates, end_date, train_days=train_days, test_days=test_days)

            fs = FilterSamples(
                Xtree_train = Xtree_train,
                ytree_train = ytree_train,
                treenames = ls.featureTreeNames,
                Xtree_test = Xtree_test,
                samples_dates_train = samples_dates_train,
                samples_dates_test = samples_dates_test,
                ytree_test = ytree_test,
                params = opt_params
            )
            _, _, s_tr, s_te = fs.run()
            s_tr = 1.0 if s_tr is None or np.isnan(s_tr) or s_tr<0.5 else s_tr
            s_te = 1.0 if s_te is None or np.isnan(s_te) or s_te<0.5 else s_te
            
        except ValueError:
            s_tr = s_te = 1.0
            
        scores_train.append(s_tr)
        scores_test.append(s_te)    
        
    logger.info(f"Trial number {trial.number}")
    logger.info(f"All scores (train) = {scores_train}")
    logger.info(f"All scores (test) = {scores_test}") 
    
    fin_score = np.mean(np.log(scores_test))
    logger.info(f"Log mean of scores (train) {np.mean(np.log(scores_train))}")
    logger.info(f"Log mean of scores (test) {np.mean(np.log(scores_test))}")
    
    return float(fin_score)

optuna.logging.enable_propagation()
study = optuna.create_study(
    direction="maximize",
)
study.optimize(objective, timeout=60*60*4)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-12 23:25:22,391] A new study created in memory with name: no-name-094bb019-ece2-4f0a-bbf4-434dfc998a97


2025-07-12 23:25:22,391 - A new study created in memory with name: no-name-094bb019-ece2-4f0a-bbf4-434dfc998a97
2025-07-12 23:25:23,113 - FilterSamples: (train) mean of y values 1.0020485267681911.
2025-07-12 23:25:23,117 - FilterSamples: (test) mean of y values 1.0070060724945438.
2025-07-12 23:25:24,230 - FilterSamples: Best init score 1.0072
2025-07-12 23:25:28,992 -   Mean target (train): 1.00716023269745
2025-07-12 23:25:28,994 -   Max distance between idces: 478
2025-07-12 23:25:29,005 -   Mean target (test): 1.045803260280321
2025-07-12 23:25:29,009 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-12 23:25:29,032 - FilterSamples: Final score (train): 1.0072
2025-07-12 23:25:29,034 - FilterSamples: Final score (test): 1.0458
2025-07-12 23:25:29,337 - FilterSamples: (train) mean of y values 0.998591182411707.
2025-07-12 23:25:29,338 - FilterSamples: (test) mean of y values 1.0484086031440085.
2025-07-12 23:25:29,975 - FilterSamples: Best init score 1.0027
2025-07-

[I 2025-07-12 23:26:33,816] Trial 0 finished with value: 0.01605928946265304 and parameters: {'lincomb_lr': 8.264627406564947e-05, 'lincomb_epochs': 900, 'lincomb_noise_std': 0.16066443670226407, 'lincomb_subsample_ratio': 0.4246607601902213, 'lincomb_sharpness': 0.4860406490848177, 'lincomb_featureratio': 0.11002191260451011, 'num_train_days': 480}. Best is trial 0 with value: 0.01605928946265304.


2025-07-12 23:26:33,816 - Trial 0 finished with value: 0.01605928946265304 and parameters: {'lincomb_lr': 8.264627406564947e-05, 'lincomb_epochs': 900, 'lincomb_noise_std': 0.16066443670226407, 'lincomb_subsample_ratio': 0.4246607601902213, 'lincomb_sharpness': 0.4860406490848177, 'lincomb_featureratio': 0.11002191260451011, 'num_train_days': 480}. Best is trial 0 with value: 0.01605928946265304.
2025-07-12 23:26:33,952 - FilterSamples: (train) mean of y values 1.0025269840969393.
2025-07-12 23:26:33,955 - FilterSamples: (test) mean of y values 1.007759952041286.
2025-07-12 23:26:34,456 - FilterSamples: Best init score 1.0096
2025-07-12 23:26:35,588 -   Mean target (train): 1.0170742172180038
2025-07-12 23:26:35,590 -   Max distance between idces: 185
2025-07-12 23:26:35,604 -   Mean target (test): 1.0198359484341737
2025-07-12 23:26:35,605 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-12 23:26:35,615 - FilterSamples: Final score (train): 1.0171
2025-07-12 23:26:35,

[I 2025-07-12 23:27:01,201] Trial 1 finished with value: 0.011843277393797352 and parameters: {'lincomb_lr': 0.002064190497438617, 'lincomb_epochs': 300, 'lincomb_noise_std': 0.00810014136033523, 'lincomb_subsample_ratio': 0.41175028900815674, 'lincomb_sharpness': 0.46854757964565996, 'lincomb_featureratio': 0.12347516643621448, 'num_train_days': 240}. Best is trial 0 with value: 0.01605928946265304.


...

[W 2025-07-13 00:10:47,972] Trial 44 failed with parameters: {'lincomb_lr': 0.0002239975303051526, 'lincomb_epochs': 700, 'lincomb_noise_std': 0.009186306082655603, 'lincomb_subsample_ratio': 0.1646861896920553, 'lincomb_sharpness': 0.2260419937017281, 'lincomb_featureratio': 0.6854364823050146, 'num_train_days': 420} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kimer\AppData\Local\Temp\ipykernel_4984\3285965210.py", line 49, in objective
    _, _, s_tr, s_te = fs.run()
                       ^^^^^^^^
  File "c:\Users\kimer\Desktop\RandomOdyssey\src\predictionModule\FilterSamples.py", line 141, in run
    lincomb_mask_train, lincomb_mask_test = self.mask_lincomb_featurereduce(
                                            ^^^^^^^^^^^^^^^^^^^^^

2025-07-13 00:10:47,972 - Trial 44 failed with parameters: {'lincomb_lr': 0.0002239975303051526, 'lincomb_epochs': 700, 'lincomb_noise_std': 0.009186306082655603, 'lincomb_subsample_ratio': 0.1646861896920553, 'lincomb_sharpness': 0.2260419937017281, 'lincomb_featureratio': 0.6854364823050146, 'num_train_days': 420} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kimer\AppData\Local\Temp\ipykernel_4984\3285965210.py", line 49, in objective
    _, _, s_tr, s_te = fs.run()
                       ^^^^^^^^
  File "c:\Users\kimer\Desktop\RandomOdyssey\src\predictionModule\FilterSamples.py", line 141, in run
    lincomb_mask_train, lincomb_mask_test = self.mask_lincomb_featurereduce(
                                            ^^^^^^^^^^^^^^^^^^^^^^^

[W 2025-07-13 00:10:47,975] Trial 44 failed with value None.


2025-07-13 00:10:47,975 - Trial 44 failed with value None.


KeyboardInterrupt: 

In [ ]:
df.to_parquet(f"notebook-optuna-{formatted_date}.parquet", index=False)